In [1]:
import requests
import json
import pandas as pd
pd.set_option('display.max_columns', None)
import os

In [2]:
def DLD_get_access_token():
    access_token_command = "curl -X POST \"https://api.dubaipulse.gov.ae/oauth/client_credential/accesstoken?grant_type=client_credentials\" -d \"client_id=pKYG2fv0ie4NfmhW7satv4AShtx5PPkx&client_secret=AA8l7Qc8mq3TsB39\""
    print("Dubai Pulse access token recieved")
    return "Bearer " + json.loads(os.popen(access_token_command).read())["access_token"]

def get_DLD_transactions(offset, DLD_access_token):
    headers = {"Authorization": DLD_access_token}
    url = "https://api.dubaipulse.gov.ae/open/dld/dld_transactions-open-api?order_by=instance_date&filter=area_id=390 AND procedure_id=11 AND property_sub_type_id=60&offset="+str(offset)
    response = requests.get(url, headers=headers).json()
    if next(iter(response)) == 'fault':
        DLD_access_token = DLD_get_access_token()
        get_DLD_transactions(offset)
    else:
        return pd.DataFrame(response["results"])

In [3]:
def loop_DLD_transactions():
    DLD_access_token = DLD_get_access_token()
    offset = 12000 #This will need to be read and updated to s3 everytime
    df = pd.DataFrame(columns=['transaction_id']) #This will need to be changed to read old transactions from s3

    DLD_API_call = get_DLD_transactions(offset, DLD_access_token)
    while sum(DLD_API_call.transaction_id.isin(df.transaction_id).astype(int)) < len(DLD_API_call):
        print("%d rows were appended to DLD transactions"%len(DLD_API_call))
        df = pd.concat([df, DLD_API_call[~DLD_API_call.transaction_id.isin(df.transaction_id)]], ignore_index=True)
        offset = offset + 1000
        DLD_API_call = get_DLD_transactions(offset, DLD_access_token)
        if DLD_API_call.empty:
            break
    
    return df

In [ ]:
df = loop_DLD_transactions()

In [4]:
DLD_access_token = DLD_get_access_token()

Dubai Pulse access token recieved


In [5]:
headers = {"Authorization": DLD_access_token}
url = "https://api.dubaipulse.gov.ae/open/dld/dld_transactions-open-api?order_by=instance_date&filter=area_id=390 AND procedure_id=11 AND property_sub_type_id=60&offset=17000"
response = requests.get(url, headers=headers).json()
# df2 = pd.DataFrame(response["results"])

In [6]:
response

{'name': 'Quota Exceeded Error',
 'errorString': 'The valid quota has been exceeded'}

In [14]:
bayut_url = "https://www.bayut.com/property-market-analysis/sale/dubai/downtown-dubai/boulevard-point/?time_since_creation=36m&beds=2"
bayut_url_response = requests.get(bayut_url)

In [16]:
print(bayut_url_response.content)

b'<!DOCTYPE html><html lang="en" dir="ltr" itemscope="" searchhistory="original" searchhistorymobile="original" itemType="http://schema.org/WebPage"><head><meta charSet="UTF-8"/><meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=0"/><link rel="dns-prefetch" href="http://LL8IZ711CS-dsn.algolia.net"/><link rel="dns-prefetch" href="https://www.googletagmanager.com"/><link rel="dns-prefetch" href="https://www.google-analytics.com"/><link rel="dns-prefetch" href="https://images.bayut.com"/><link rel="apple-touch-icon" sizes="180x180" href="/assets/apple-touch-icon.7919a66ff6f58dbfb1d2e33cb19b5a80.png"/><link rel="icon" type="image/png" sizes="16x16" href="/assets/favicon-16x16.0e9b01e26e8e124b4b030829ed6e5bfb.png"/><link rel="icon" type="image/png" sizes="32x32" href="/assets/favicon-32x32.20ce04f8a637e94fd0dc5ca4eeab1dce.png"/><link rel="manifest" href="/assets/cfd539f6f9fa15a35156f806828fe844.json"/><link rel="mask-icon" href="/assets/safari-pinned-tab.f33